In [40]:
import torch
import torchvision
from torchvision import transforms, models
import numpy
import torch.utils.data as data
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
import time

## 预处理
1. 设置参数 
2. 图片的路径 
3. 定义了数据增强, 数据增强包括 图片的大小调整， 居中， 归一化，转换成Tensor的格式
4. 定义了dataset 把数据加载到dataset中 并应用到数据增强中
5. 把dataset加载到dataloader中


In [64]:
epoch = 20 
batch_size = 64
learning_rate = 0.01
train_datapath = '/Users/mac/Desktop/traindata/'
test_datapath = '/Users/mac/Desktop/testdata/'
momentum = 0.9
step_size = 7
gamma = 0.1


In [52]:
transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    
])


In [53]:
train_data = torchvision.datasets.ImageFolder(root=train_datapath, transform=transformer)
test_data = torchvision.datasets.ImageFolder(root=test_datapath, transform=transformer)


In [54]:
train_dataloader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = data.DataLoader(test_data, batch_size=batch_size, shuffle=True)


## 迁移学习 

In [55]:
class_names = train_data.classes
class_names


['1',
 '11',
 '12',
 '17',
 '22',
 '23',
 '24',
 '27',
 '29',
 '30',
 '32',
 '34',
 '36',
 '37',
 '41',
 '43',
 '48',
 '50',
 '51',
 '56',
 '58',
 '60',
 '63',
 '65',
 '66',
 '7',
 '72',
 '79',
 '81',
 '82',
 '83',
 '84',
 '87',
 '89',
 '9']

In [56]:
len(class_names)

35

In [57]:
device = torch.device("cpu")

In [58]:
model = models.resnet50(pretrained=True).to(device)

In [59]:
for param in model.parameters():
    param.require_grad = False

In [60]:
model.fc = torch.nn.Sequential(
               torch.nn.Linear(2048, 128),
               torch.nn.ReLU(inplace=True),
               torch.nn.Linear(128, len(class_names))).to(device)

## 换了RESNET的FC层

In [61]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
dataloaders = {'train': train_dataloader, 'val': test_dataloader}

## 设置了优化器 损失函数 scheduler 快到损失最优值的时候降低学习率

In [69]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [73]:
train_model(model, criterion, optimizer, scheduler, num_epochs=epoch)

Epoch 0/19
----------


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mac/Desktop/traindata/30/6bc4039ca551781e.JPG'